In [7]:
'''
Created on July 8, 2019    
@author: SungWook Yoo 
'''

'\nCreated on July 8, 2019    \n@author: SungWook Yoo \n'

# Selection Algorithm 
in oder to simplify the problem, <font color=red>assume that array values should be **distinct**</font>.

In [6]:
import numpy as np
# QuickSelect: pick k th smallest element.
# assume that array values should be distinct.
# it takses average O(n), worst O(n^2) time.
# This is because pivot determines reculsive call deviding ratio! 
def QuickSelect(a, p, r, k):
    if p == r:
        return a[p]
    else:
        i = np.random.randint(p, r+1)
        a[r], a[i] = a[i], a[r]
        # randomized partition with pivot as a[r]
        i = p - 1
        for j in range(p, r):
            if a[j] <= a[r]:
                i += 1 
                a[i], a[j] = a[j], a[i]
        a[i+1], a[r] = a[r], a[i+1]
        q = i+1
        # divide and conquer for QuickSelect
        i = q - p + 1
        if k <= i:
            return QuickSelect(a, p, q, k)
        else:
            return QuickSelect(a, q+1, r, k-i)

In [92]:
import random
a = random.sample(range(1, 100), 10)
a 

[62, 86, 44, 69, 78, 94, 48, 47, 99, 15]

In [23]:
QuickSelect(a, 0, len(a) - 1, 9)

76

average time complexity analysis
$$
T(n) = T(n/2) + O(n) = O(n)
$$
However, worst case time complexity is $O(n^2)$  
This is because pivot determines dividing ratio.  
so, in a worst case, 
$$
T(n) = T(n-1) + O(n) = O(n^2)
$$

# Better Algorithm 
[PseudoCode] kthSmallest using finding Median of Median and tranformed QickSelect algorithm [link](https://www.geeksforgeeks.org/kth-smallestlargest-element-unsorted-array-set-3-worst-case-linear-time/)
```C++
kthSmallest(arr[0..n-1], k)
1) Divide arr[] into ⌈n/5⌉ groups where size of each group is 5 
   except possibly the last group which may have less than 5 elements.  

2) Sort the above created ⌈n/5⌉ groups and find median 
   of all groups. Create an auxiliary array 'median[]' and store medians 
   of all ⌈n/5⌉ groups in this median array.

// Recursively call this method to find median of median[0..⌈n/5⌉-1]
3) medOfMed = kthSmallest(median[0..⌈n/5⌉-1], ⌈n/10⌉)

4) Partition arr[] around medOfMed and obtain its position.
     pos = partition(arr, n, medOfMed)

5) If pos == k return medOfMed 
6) If pos > k return kthSmallest(arr[l..pos-1], k) 
7) If pos < k return kthSmallest(arr[pos+1..r], k-pos+l-1)
```

In [99]:
""" helper functions """
# assume that r-p+1 <= 5
def findMedian(a, p, r):
    L = []
    for i in range(p, r+1):
        L.append(a[i])
    L.sort()
    return L[(r-p+1)//2]

def partition(a, p, r, x):
    
    # we should find out medOfmed's index i value in the a[p..r]
    # swap a[i], a[r] in order to make a[r] as a pivot 
    for i in range(p, r+1): 
        if a[i] == x: 
            a[i], a[r] = a[r], a[i]
            break
    
    i = p - 1
    for j in range(p, r):
        if a[j] <= a[r]:
            i += 1 
            a[i], a[j] = a[j], a[i]
    a[i+1], a[r] = a[r], a[i+1]
    return i+1

In [100]:
def KthSmallest(a, p, r, k):
    # divide A into floor(n/5) groups
    # create median array with size floor(n/5)
    n = r - p + 1
    median = []
    i = 0
    while i < n//5:
        # 5 element can be assigned for each group
        median.append(findMedian(a, p+5*i, p+5*i+4))
        i += 1
    # if last group has n%5 (remainder) elements
    if i*5 < n:
        median.append(findMedian(a, p+5*i, p+5*i+(n%5-1)))
        i += 1
    # so, at this time i value means floor(5/n)
    if i == 1:
        # if median has only one elements, the medOfmed should be median[0]
        medOfmed = median[i-1]
    else:
        # reculsively medOfmed can be found. 
        # Because median array is generated each recursion, i value should be shrunk more and more 
        medOfmed = KthSmallest(median, 0, i-1, i//2)
    
    # at this bottom line, medOfmed can be determined 
    # if we use the pivot as medofmed value, the number of sorted elements can be 3(floor(n/5)/2 - 2)
    q = partition(a, p, r, medOfmed)
    # i value means medOfmed's rank in a[...] array 
    i = q - p + 1 
    if i == k:
        # if partitioned pivot is the kth Smallest element
        return a[q]
    elif i > k:
        return KthSmallest(a, p, q-1, k)
    else:
        return KthSmallest(a, q+1, r, k-i)

In [116]:
a = random.sample(range(1, 10000000), 100000)
# print(a)
n = len(a)  
k = 3
%time print("1. worst case O(n^2), average O(n) select algorithm's output: ", QuickSelect(a, 0, n - 1, k))
%time print("2. worst case O(n) select algorithm's output: ", KthSmallest(a, 0, n - 1, k))

1. worst case O(n^2), average O(n) select algorithm's output:  128
CPU times: user 125 ms, sys: 0 ns, total: 125 ms
Wall time: 126 ms
2. worst case O(n) select algorithm's output:  128
CPU times: user 312 ms, sys: 0 ns, total: 312 ms
Wall time: 326 ms


$T(n/5)$ means reculsive call when finding medOfmed value  
when we use medOfmed as a pivot, after partitioning,  
assure that at least $3((n/5)/2 - 2)$ elements must be less or larger than medOfmed.  
(where, -2 term means except for medOfmed group and last group.)  
so, next reculsive call for KthSmallest selcect array size is $n - 3((n/5)/2 - 2) = 7n/10 + 6$ 
$$
\begin{aligned}
T(n) &= T(n/5) + T(n - 3((n/5)/2 - 2)) + O(n)  \\
    &= T(n/5) + T(7n/10 + 6) + O(n) \\
    &= O(n)
\end{aligned}
$$

# Better Quick Sort Algorithm 
If we find out medOfmed in $O(n)$ time, and the medOfme as a pivot.  
at worst case, Quick Sort time complexity $O(nlogn)$

In [118]:
def QuickSort(a, p, r):
    if p >= r:
        return 
    med = KthSmallest(a, p, r, (r-p+1)//2)
    q = partition(a, p, r, med)
    
    QuickSort(a, p, q-1)
    QuickSort(a, q+1, r)

In [119]:
QuickSort(a, 0, n-1)

In [122]:
# this value should be consist with above values: QuickSelect(a, 0, n - 1, k), KthSmallest(a, 0, n - 1, k)
a[k-1]

128